In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision import models, datasets, transforms
from torchvision.utils import make_grid
from torchvision.datasets.utils import download_url

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import tarfile
import time
from tqdm.auto import tqdm
%matplotlib inline

In [2]:
def set_requires_grad(model, value=False):
    for param in model.parameters():
        param.requires_grad = value

In [3]:
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

In [4]:
num_classes = 10
input_size = 224
batch_size = 64

In [5]:
model = models.resnet18(pretrained=True)
# set_requires_grad(model, False)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_classes)

In [6]:
normalize = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [7]:
dataset_url = 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160.tgz'
db = download_url(dataset_url,'.')

Using downloaded and verified file: ./imagenette-160.tgz


In [21]:
# Extract from archive
with tarfile.open('./imagenette-160.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
    
# Look into the data directory
data_dir = './data/imagenette-160'
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['val', 'train']
['n02979186', 'n03888257', 'n03417042', 'n01440764', 'n03425413', 'n03028079', 'n02102040', 'n03000684', 'n03445777', 'n03394916']


In [8]:
!ls 'data/imagenette-160'

train  val


In [8]:
# PyTorch datasets
train_ds = ImageFolder('data/imagenette-160/train', normalize)
valid_ds = ImageFolder('data/imagenette-160/val', normalize)

In [9]:
imagenette_trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
imagenette_testloader = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
imagenette_loaders = {'train': imagenette_trainloader, 'val': imagenette_testloader}


In [10]:
cifar_trainset = datasets.CIFAR10(root='./cifar10', train=True,
                            download=True, transform=normalize)
cifar_trainloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

cifar_testset = datasets.CIFAR10(root='./cifar10', train=False,
                           download=True, transform=normalize)
cifar_testloader = torch.utils.data.DataLoader(cifar_testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

cifar_loaders = {'train': cifar_trainloader, 'val': cifar_testloader}
cifar_classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
model = model.to(device)
pretrain_optimizer = torch.optim.SGD(model.fc.parameters(),
                                     lr=0.001, momentum=0.9)
train_optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
def train_model(model, dataloaders, criterion, optimizer,
                phases, num_epochs=3):
    start_time = time.time()

    acc_history = {k: list() for k in phases}
    loss_history = {k: list() for k in phases}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                # print("labels: ", labels)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double()
            epoch_acc /= len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss,
                                                       epoch_acc))
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                        time_elapsed % 60))

    return model, acc_history

Проверяем качество исходной модели на imagenette

In [15]:
train_model(model, imagenette_loaders, criterion, train_optimizer,
            phases=['train', 'val'], num_epochs=1)

Epoch 0/0
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.9973 Acc: 0.8399


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.1296 Acc: 0.9720

Training complete in 0m 53s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [16]:
orig_fc_layer = model.fc

In [17]:
set_requires_grad(model, False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

In [18]:
model = model.to(device)

Дообучим последний слой модели решать датасет CIFAR10

In [19]:
train_model(model, cifar_loaders, criterion, pretrain_optimizer,
            phases=['train', 'val'], num_epochs=2)

Epoch 0/1
----------


  0%|          | 0/782 [00:00<?, ?it/s]

train Loss: 2.5116 Acc: 0.1116


  0%|          | 0/157 [00:00<?, ?it/s]

val Loss: 2.5137 Acc: 0.1148

Epoch 1/1
----------


  0%|          | 0/782 [00:00<?, ?it/s]

train Loss: 2.5116 Acc: 0.1127


  0%|          | 0/157 [00:00<?, ?it/s]

val Loss: 2.5073 Acc: 0.1145

Training complete in 3m 43s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

Дообучим все слои на датасете CIFAR-10

In [20]:
set_requires_grad(model, True)
train_model(model, cifar_loaders, criterion, train_optimizer,
            phases=['train', 'val'], num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/782 [00:00<?, ?it/s]

train Loss: 0.7479 Acc: 0.7642


  0%|          | 0/157 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
    assert self._parent_pid == os.getpid(), 'can only test a child process'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
AssertionError: can only test a child process
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

val Loss: 0.2066 Acc: 0.9368

Epoch 8/9
----------


  0%|          | 0/782 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

KeyboardInterrupt: ignored

На валидационном датасете CIFAR10 получили качество 0.94. Попробуем вернуть оригинальный после5дний слой Imagenette и проверить, что стало с качеством

In [21]:
model.fc = orig_fc_layer

In [22]:
train_model(model, imagenette_loaders, criterion, train_optimizer,
            phases=[ 'val'], num_epochs=1)

Epoch 0/0
----------


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 1.5450 Acc: 0.5800

Training complete in 0m 2s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

На валидационном датасете получили 0.58 - качество ожидаемо просело. Попробуем обучить только последний слой на Imagenette и проверить, удалось ли добиться исходного качества 

In [23]:
set_requires_grad(model, False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, model.fc.out_features)

In [25]:
model = model.to(device)

In [26]:
train_model(model, imagenette_loaders, criterion, train_optimizer,
            phases=['train', 'val'], num_epochs=5)

Epoch 0/4
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 7.2136 Acc: 0.0000


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 7.2371 Acc: 0.0000

Epoch 1/4
----------


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
    if w.is_alive():
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
Traceback (most recent call last):
    if w.is_alive():
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3daa69f290>
  File 

train Loss: 7.2134 Acc: 0.0000


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 7.2496 Acc: 0.0000

Training complete in 3m 16s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

Исходного качества не добились, поскольку для классификации важны признаки из предыдущих слоев, которые мы не переобучали.